In [1]:
batch_size = 64

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


In [3]:
mnist_dataset_train = datasets.MNIST(root="dataset", train=True, download=True, transform=ToTensor())
mnist_dataset_test = datasets.MNIST(root="dataset", train=False, download=True)


In [4]:
train_dataloader = DataLoader(mnist_dataset_train, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(mnist_dataset_test, batch_size=batch_size, shuffle=False)